# <center> Web Scraping </center>

**Objectif :** 

Récupérer des informations de qualité pour chaque ouvrage et les inscrire dans un csv.

**Sites à parcourir pour récupérer des informations :**

- Paris libraire : https://www.parislibrairies.fr/livre/ [ISBN] : format JSON = inaccessible en webscraping (?)
        
- Librairies indépendantes : https://www.librairiesindependantes.com/product/ [ISBN] (récupère infos facilement, par contre le titre porte une maj à chaque mot)
        
- Les libraires.fr : https://www.leslibraires.fr/livre/ [nombre_incrémenté]



**Sites pour récupérer seulement des informations :**
Incrémentation des pages impossible du fait de l'url, le web scraping est donc impossible. Cependant, les sites en question peuvent être utiles pour récupérer des informations car le code de la page est bien construit.

- Gibert.com 
- Momox.shop



# Les libraires indépendantes (test)

- nécessite une liste d'ISBN, les données sont bien décrites cependant le titre n'a pas de majuscule pour les noms propres (dû au fait de faire un capitalize() sur le titre recueilli pour enlever les maj à chaque mot, exemple : "C'est La Ville De Brest" devient ainsi : "C'est la ville de brest")

### Pour 1 seul ISBN :

In [6]:
from bs4 import BeautifulSoup
import requests

ISBN = 9782020047135 #9782020789516 
url = 'https://www.librairiesindependantes.com/product/'+str(ISBN)
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')


# liste récupérant les informations 
liste = []


# récupérer l'auteur
auteur = soup.find("h2")
liste.append(auteur.text)

# récupérer le titre
titre =soup.find("h1")
liste.append(titre.text)

# Récupérer l'édition
edition = soup.find_all("h3")
liste.append(edition[1].text)

# Récupérer la date
date = soup.findAll("dd")
# on récupère la date dans une variable temporaire
var_temp = date[2].text
# on split la date et on ne récupère que l'année
split = var_temp.split()
date = split[-1]
# on ajoute la date (format : int) à la liste
# liste.append(int(date))
liste.append(date)

# Ajouter l'ISBN
liste.append(str(ISBN))

# -----------------------------------------
# Résultat
print(liste)

['Henri-Irénée Marrou', 'Décadence Romaine Ou Antiquité Tardive ? , Iiie-Vie Siècle', 'Points', '1977', '9782020047135']


## Web Scraping pour plusieurs ISBN

In [17]:
from bs4 import BeautifulSoup
import requests
import csv
from tqdm import tqdm
# --------------------------------------------------------------

# ISBN (indexe les pages)
liste_ISBN = [9782930088938, 9782209059478, 9782749901312  ]

# récupère chaque liste d'infos sur 1 livre
liste_totale = []

compteur = 0

while compteur < len(liste_ISBN):

    # Web scraping pour chaque livre dans la liste_ISBN
    for livre in liste_ISBN :

        try : 
            url = 'https://www.librairiesindependantes.com/product/'+str(livre)
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')

            # liste récupérant les informations 
            liste = []


            # récupérer l'auteur
            auteur = soup.find("h2")
            liste.append(auteur.text)

            # récupérer le titre
            titre = soup.find("h1")
        #     liste.append(titre.text)
        # capitaliser le titre (enlever les maj chaque mot)
            titre = titre.text
            titre = titre.capitalize()
            liste.append(titre)

            # Récupérer l'édition
            edition = soup.find_all("h3")
            liste.append(edition[1].text)

            # Récupérer la date
            date = soup.findAll("dd")
            # on récupère la date dans une variable temporaire
            var_temp = date[2].text
            # on split la date et on ne récupère que l'année
            split = var_temp.split()
            date = split[-1]
            # on ajoute la date (format : int) à la liste
            # liste.append(int(date))
            liste.append(date)

            # Ajouter l'ISBN
            liste.append(str(ISBN))

            # On ajoute la liste à la liste totale (elle servira à écire dans le csv)
            liste_totale.append(liste)

            
            compteur += 1

        except :
            print(livre, "ISBN incorrect")
            compteur += 1
    
# Affichage des résultats 
print("Informations : ", liste_totale)


'''-----------------------------
Intégration dans un csv
-------------------------------'''

# ## Création du csv (s'il n'existe pas)
# header = ['Auteur(s)', 'Titre', 'Edition', 'Date', 'ISBN']
# with open(r'web_scrap_ameliore.csv', 'w') as f:
#     writer = csv.writer(f)
#         # write the header
#     writer.writerow(header)
#     print("fichier créé")


# # insérer les informations pour chaque livre (à partir de la liste totale)
# with open(r'web_scrap_ameliore.csv', 'a', encoding='UTF8') as f:
#         for x in tqdm(liste_totale) :
#             writer = csv.writer(f)
#             writer.writerow(x)
        
#         # message de réussite
#         print("Insertion des données réussies.")

9782930088938 ISBN incorrect
9782209059478 ISBN incorrect
9782749901312 ISBN incorrect
Informations :  []


'-----------------------------\nIntégration dans un csv\n-------------------------------'

### Lecture du fichier

In [16]:
import pandas as pd
df = pd.read_csv("web_scrap_ameliore.csv")
df

,Auteur(s),Titre,Edition,Date,ISBN
0,Karine Duchochois,"Moi, karine innocente et cassée",Plon,2004,9782259201520
1,Florence Aubenas,"La méprise. l'affaire d'outreau, l'affaire d'o...",Seuil,2005,9782259201520
2,Chérif Delay,"Je suis debout, l'ainé des enfants d'outreau s...",Le Cherche Midi,2011,9782259201520
3,Henri-Irénée Marrou,"Décadence romaine ou antiquité tardive ? , iii...",Seuil,1977,9782020047135
4,Henri-Irénée Marrou,"L'eglise de l'antiquité tardive (303-604), 303...",Seuil,1985,9782020047135


### Modifications 

In [15]:
df.loc[4, "Edition"] = "Seuil"

df
# Sauvegarde du csv 
df.to_csv("web_scrap_ameliore.csv", index=False)

In [ ]:
# # enregistrer les modifications
# df.to_csv("web_scrap_ameliore.csv", index=False)

---

# Les libraires.fr (test)

In [1]:
import csv
## Création du csv (s'il n'existe pas)
header = ['Auteur(s)', 'Titre', 'Edition', 'Date', 'ISBN']
with open(r'test2.csv', 'w') as f:
    writer = csv.writer(f)
        # write the header
    writer.writerow(header)
    print("fichier créé")

fichier créé


In [7]:
import requests
from bs4 import BeautifulSoup
import csv
# from tqdm import tqdm
from tqdm.notebook import tqdm_notebook

nb_livres = 5000



nombre = 200000


liste_totale = []

for livre in tqdm_notebook(range(nb_livres)) : 

    erreur = 0
    try : 
        url = 'https://www.leslibraires.fr/livre/' + str(nombre)
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        liste = []

        # auteur 
        auteur = soup.find("h2")
        auteur = auteur.text.strip('\n')
        liste.append(auteur)

        # titre
        titre = soup.find("h1")
        titre = titre.text.strip('\n')
        liste.append(titre)

        # edition
        edition = soup.find("h3")
        edition = edition.text.strip('\n')
        liste.append(edition)

        # date
        date = soup.findAll("dd")
        # on récupère la date dans une variable temporaire
        var_temp = date[5].text
        # on split la date et on ne récupère que l'année
        split = var_temp.split("/")
        date = split[-1]
        # # on ajoute la date (format : int) à la liste
        # liste.append(int(date))
        liste.append(date)

        # ISBN
        # Récupérer l'isbn
        info = soup.findAll("dd")
        # on récupère l'isbn dans une variable temporaire
        isbn = info[2].text
        liste.append(isbn)

        liste_totale.append(liste)

        nombre += 1
        
    except :

        nombre += 1
        erreur += 1


print("Nb d'erreurs :", erreur)
    
# insérer les informations pour chaque livre (à partir de la liste totale)
with open(r'test2.csv', 'a', encoding='UTF8') as f:
        for x in tqdm_notebook(liste_totale) :
            writer = csv.writer(f)
            writer.writerow(x)
        
        # message de réussite
        print("Insertion des données réussies.")

  0%|          | 0/5000 [00:00<?, ?it/s]

Nb d'erreurs : 0


  0%|          | 0/4628 [00:00<?, ?it/s]

Insertion des données réussies.


## Gestion des infos (suppression, modification...)

In [1]:
# import pandas as pd
# df = pd.read_csv('lib_webscrap_recup.csv')
# df

### 1- Sélectionner des lignes dans le df

In [453]:
df.loc[1650:1700, :]

,Auteur(s),Titre,Edition,Date,ISBN
1600,"Corbeil, Jean-Claude",Oxford visual five language dictionary,Oxford University Press,2006,9780199205981
1601,Avraham B. Yehoshua,Trois Jours Et Un Enfant,Denoël,1974,9782207281871
1602,Diderot Denis,Correspondance,Les Éditions de Minuit,1958,9782707304391
1603,De Denise Crolle-Terzaghi,Petits Biscuits pour toutes les fêtes,Illustrations de Nathalie Schott,2009,9782737336751
1604,pratt hugo,Litho saint-ex (5 + 1 gratis),Casterman,1995,9782203934351
1605,De Friedrich Nietzsche,Contribution A La Genealogie De La Morale,L'Harmattan,2006,9782296000421
1606,Sauvé des eaux,1 - Paul et Colombe - Tome 1 : Sauvé des eaux,Triomphe,2005,9782843782671
1607,De Pierre Bruno,Papiers psychanalytiques\r\n\r\n2 - La passe,Presses universitaires du Midi,2003,9782858167081
1608,Michel Matana,Maisons de bord de mer,Alternatives,1998,9782862271071
1609,De Agustina Bessa Luis,Les Terres du risque,Anne-Marie Métailié,1996,9782864242291


In [452]:
# suppression des lignes inutiles
df.drop([1650], axis=0, inplace=True)

df.to_csv("web_scrap_leslibraires.csv", index=False)

df = pd.read_csv("web_scrap_leslibraires.csv")


#### Modification des données

In [242]:
# Visualiser la donnée à modifier
df.loc[455]

Auteur(s)                     David S. Landes
Titre        Richesse et pauvreté des nations
Edition                          Albin Michel
Date                                     2000
ISBN                            9782226110381
Name: 455, dtype: object

In [241]:
df.loc[455, "Date"] = 2000


df
# Sauvegarde du csv 
df.to_csv("web_scrap_leslibraires.csv", index=False)

# # affichage du df
# df.tail(10)

In [96]:
# Supprimer les doublons
df.drop_duplicates(inplace=True)

In [110]:
# Supprimer une ligne
df.drop(index=5, axis=0, inplace=True)
df

,Auteur(s),Titre,Edition,Date,ISBN
6,Approche linguistico-mathématique,La Sémiotique formelle du folklore,Contributions de Tudor Bălănescu,1978,9782252020111
7,De Paul Beaulier,Pauline,L'Harmattan,2005,9782747579841
8,De Jacques Labescat,Guide des examens complémentaires,Lamarre,2008,9782757301821
9,De Stéphane Benhamou,Les années 60 pour les nuls,First,2009,9782754006101


In [111]:
# Sauvegarde du dataframe
df.to_csv("web_scrap_leslibraires.csv", index=False)

In [112]:
df = pd.read_csv("web_scrap_leslibraires.csv")
df

,Auteur(s),Titre,Edition,Date,ISBN
0,Approche linguistico-mathématique,La Sémiotique formelle du folklore,Contributions de Tudor Bălănescu,1978,9782252020111
1,De Paul Beaulier,Pauline,L'Harmattan,2005,9782747579841
2,De Jacques Labescat,Guide des examens complémentaires,Lamarre,2008,9782757301821
3,De Stéphane Benhamou,Les années 60 pour les nuls,First,2009,9782754006101


### Récupérer infos pour 1 livre

In [78]:
liste = []


# auteur 
auteur = soup.find("h2")
auteur = auteur.text.strip('\n')
liste.append(auteur)

# titre
titre = soup.find("h1")
titre = titre.text.strip('\n')
liste.append(titre)

# edition
edition = soup.find("h3")
edition = edition.text.strip('\n')
liste.append(edition)

# date
date = soup.findAll("dd")
# on récupère la date dans une variable temporaire
var_temp = date[5].text
print(var_temp)
# on split la date et on ne récupère que l'année
split = var_temp.split("/")
date = split[-1]
# # on ajoute la date (format : int) à la liste
# liste.append(int(date))
liste.append(date)

# ISBN
# Récupérer l'isbn
info = soup.findAll("dd")
# on récupère l'isbn dans une variable temporaire
isbn = info[2].text
liste.append(isbn)

25/01/1999


In [79]:
liste

['De Collectif', 'Qui est dieu ?', 'Guy Trédaniel', '1999', '9782844450531']